In [103]:
import pandas as pd
import json

In [43]:
file_path = "milanote-activities.csv"
data = pd.read_csv(file_path)

In [44]:
print("Preview of the dataset:")
data.head()

Preview of the dataset:


,id,createdAt,campaignId,offerId,sender,message,type,userId,stageId,input,dateInput,attachments,submitted,approved,feedback,feedbackAttachments
0,8593,2024-11-25T19:01:26.423Z,021b7fc2-b4cb-4d6f-81f9-04e3e0cba46f,NaN,Milanote,"Approved Draft Video With Feedback:\nHey shar,...",NaN,NaN,5,https://drive.google.com/file/d/1ogbYpjX4FAx2k...,2024-11-28T00:00:00.000Z,NaN,True,True,"Hey shar, \n\nThe integration segment looks gr...",NaN
1,8593,2024-11-25T19:01:26.423Z,021b7fc2-b4cb-4d6f-81f9-04e3e0cba46f,NaN,shar,Submitted Draft Video,NaN,NaN,5,https://drive.google.com/file/d/1ogbYpjX4FAx2k...,2024-11-28T00:00:00.000Z,NaN,True,True,"Hey shar, \n\nThe integration segment looks gr...",NaN
2,8198,2024-11-19T05:24:59.336Z,021b7fc2-b4cb-4d6f-81f9-04e3e0cba46f,NaN,Milanote,Approved Draft Script With Feedback:\nHey shar...,NaN,NaN,4,[hi Himika/Milanote team! :) if you need addit...,NaN,NaN,True,True,"Hey shar, \n\nThe script looks great and bring...",NaN
3,8198,2024-11-19T05:24:59.336Z,021b7fc2-b4cb-4d6f-81f9-04e3e0cba46f,NaN,shar,Submitted Draft Script,NaN,NaN,4,[hi Himika/Milanote team! :) if you need addit...,NaN,NaN,True,True,"Hey shar, \n\nThe script looks great and bring...",NaN
4,6600,2024-10-18T21:59:59.724Z,021b7fc2-b4cb-4d6f-81f9-04e3e0cba46f,NaN,Milanote,Approved Video Topic & Milanote Use Case With ...,NaN,NaN,1,changing video topic to be the process of maki...,NaN,NaN,True,True,"Hey shar,\n\nThat sounds great - I am happy to...",NaN


Check for emptry rows

In [45]:
empty_rows = data[data.isnull().all(axis=1)]
print(f"\nNumber of empty rows: {len(empty_rows)}")
if not empty_rows.empty:
    print("Empty rows detected:")
    print(empty_rows)


Number of empty rows: 0


Check for emptry columns

In [46]:
empty_columns = [col for col in data.columns if data[col].isnull().all()]
print(f"\nNumber of empty columns: {len(empty_columns)}")
if empty_columns:
    print("Empty columns detected:")
    print(empty_columns)


Number of empty columns: 5
Empty columns detected:
['offerId', 'type', 'userId', 'attachments', 'feedbackAttachments']


In [47]:
# Drop empty columns
data = data.dropna(axis=1, how='all')

**stageId** specifies which stage of the process
- 5: video submission and approval
- 4: script submission and approval
- 1: topic submission and approval

In [48]:
data["stageId"].value_counts()

stageId
5    206
4    153
1     96
Name: count, dtype: int64

### cleaning all submissions to clearly see decisions, inputs and feedback for each campaign id

In [64]:
# only keeping rows that have some rejection or approval decision
data_new = data[data["message"].str.lower().str.split().str[0].isin(["approved", "rejected"])]

In [68]:
data_new["decision"] = data_new["message"].str.lower().str.split().str[0].map({
    "approved": 1,
    "rejected": 0
})

/var/folders/j0/cdb0d43n50766_hv14h2q26m0000gn/T/ipykernel_12140/335158838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new["decision"] = data_new["message"].str.lower().str.split().str[0].map({


In [ ]:
data_new.head()

generating dataframe just for scripts

In [ ]:
data_scripts = data_new[data_new["stageId"] == 4]

In [74]:
data_scripts["decision"].value_counts()

decision
1    65
0    10
Name: count, dtype: int64

In [ ]:
data_scripts

## testing feedback key point extraction

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    api_key="insert key"
)


### Method 1: Extracting key points from the brief

In [467]:
script = """
Every now and then I like to compare my old art and the art I make currently to see if I’m improving, and also critic myself and find areas in my art I can improve on and get better. But it’s usually difficult organizing every drawing together and putting side by side on the same page so i reached out to the heavens for help and the art gods blessed me with milanote, the sponsor of today’s video. Milanote is an interactive tool that helps you organize all the different files for your CREATIVE projects on one platform so accessing them can be easier. Milanote allows you to collect and organize pictures, videos, notes and text, and arrange them into different boards using any of their built in templates that make it easy to start off your creative projects. You can also create several tasks for different things you want to study, and tick them when you’re done. For this instance, I’ll start by creating a general board where I’ll layout a broad overview that covers everything concerning the project I’m about to work on. next I’ll move on to place my references in a sub board and place my old art and current art in separate boards so I know where to find each of them. Then I create a new board where I place both drawings side by side so I can compare them, and also make notes on things I want to improve on or change. I like to use milanote’s web clipper tool when gathering screenshots from other artists that draw in a style I admire, and then save every image or screenshot I find useful In the board. 
 This also works well when you’re collaborating with other artists and working on the same project, so everyone has a good understanding of what the project is about and has access to the same references all at once since everything is properly organized in the same place.
The kind folks at milanote are giving free access with no time limit to everyone who signs up for a free account using the link in the description. So head over to https://milanote.com/mohammedagbadi to try it out for free with no time limit. All the links will be in the description, and thank you to milanote for sponsoring this video.
"""

In [ ]:
print(script)

In [ ]:
brief_guidelines = """
Brand Safety Guidelines
Milanote does not sponsor video content that includes any of the following:
- Explicit adult themes, language, or imagery
- Polarising political topics
- Personal attacks or targeted harassment
- Unverified theories presented as facts

Script flow:
1. Introduce sponsorship
2. Describe what Milanote is
3. Describe how your audience could use Milanote
4. Describe how you use Milanote
5. Describe how to collaborate and share work
6. Call to action

What to avoid in your script:
- **Mentioning YouTube Planning:** Since many viewers may not be YouTubers, discussing how you use Milanote for YouTube content planning may not be relatable; such as YouTube thumbnail, title, and posting schedule. Instead, discuss how it helps you as a content creator with your filmmaking, writing process, and research planning.
- **Neglecting an Introduction:** Avoid diving straight into your project without first explaining what Milanote is. Briefly describe “Milanote is a tool for organizing your creative projects.”
- **Sign-Up Instructions:** Refrain from mentioning 'downloading' Milanote. Instead, direct users to 'sign up for free.'
- **Forgetting Call to Action:** Remember to encourage your viewers to sign up using the link in the description.
"""

In [ ]:
# chained approach to generating feedback 3, use this

# 1. Extract key sections from the brief
system_prompt_extract = """
You are an AI assistant that extracts the key points from a creative brief to be used to generate feedback on submitted scripts.
Each point should be a single string which details what needs to be checked for.
Each point should be put into a JSON structure as shown in the following example:

Input:
Animal Guidelines
Milanote does not sponsor video content that includes:
- pictures or videos of animals

Plant Guidelines
Milanote recommends that sponsor video content includes:
- pictures or videos of plants

Output:
{
  "Key points": [
    "Check that the script does not contain any pictures or videos of animals.",
    "Check that the script contains pictures or videos of plants."
    ]
}

"""
user_prompt_extract = f"""
Creative Brief: {brief_guidelines}
"""

prompt = [
        {"role": "system", "content": system_prompt_extract},
        {"role": "user", "content": user_prompt_extract}
    ]
response_extract = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=prompt
)
key_points = response_extract.choices[0].message.content.strip()
print(key_points)
print()



{
  "Key points": [
    "Check that the script avoids explicit adult themes, language, or imagery.",
    "Check that the script avoids polarising political topics.",
    "Check that the script avoids personal attacks or targeted harassment.",
    "Check that the script avoids unverified theories presented as facts.",
    "Check that the script does not mention YouTube Planning, but instead focuses on content creation like filmmaking, writing process, and research planning.",
    "Check that the script includes an introduction explaining what Milanote is as a tool for organizing creative projects.",
    "Check that the script refrains from mentioning 'downloading' Milanote and directs users to 'sign up for free'.",
    "Check that the script includes a call to action encouraging viewers to sign up using the link in the description."
  ]
}



In [ ]:
key_points_json = json.loads(key_points)
feedbacks = []
for key, value in key_points_json.items():
    for point in value:
        print(point)
        system_prompt_compare = f"""
        Guideline:{point}. 
        Keep all feedback concise and within 1 sentence. Don't hallucinate anything.
        If the script already follows the guideline, don't repeat the guideline
        """
        user_prompt_compare = f"""

        Script: {script}

        """
        
        prompt = [
            {"role": "system", "content": system_prompt_compare},
            {"role": "user", "content": user_prompt_compare}
        ]
        response_compare = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=prompt
        )
        feedback = response_compare.choices[0].message.content.strip()
        # print(feedback)
        # print()
        feedbacks.append(feedback)


In [ ]:

# 3. Combine the feedback messages
feedback_combined_prompt = f"""
Combine the following feedback messages into a single, cohesive feedback report. Ensure no information is repated and put it into one paragraph.

Feedbacks:
{json.dumps(feedbacks, indent=2)}

Output the combined feedback.
"""
response_combined = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": feedback_combined_prompt}]
)

combined_feedback = response_combined.choices[0].message.content.strip()
print(combined_feedback)

The script is free of adult themes, language, or imagery, neutral and avoids polarizing political topics, refrains from personal attacks or targeted harassment, stays on track avoiding unverified theories presented as facts, and focuses on content creation through organizing and improving art projects using Milanote, aligning with the guideline of emphasizing filmmaking, writing process, and research planning rather than YouTube planning. Additionally, the script effectively includes an introduction explaining Milanote as a tool for organizing creative projects, showcases its features and benefits, refrains from mentioning 'downloading' Milanote and directs users to 'sign up for free' to try it out with no time limit, and includes a call to action encouraging viewers to sign up using the link in the description.


In [ ]:
# putting the chained approach 3 into functions

def extract_key_points(brief_guidelines):
    system_prompt_extract = """
    You are an AI assistant that extracts the key points from a creative brief to be used to generate feedback on submitted scripts.
    Each point should be a single string which details what needs to be checked for.
    Each point should be put into a JSON structure as shown in the following example:

    Input:
    Animal Guidelines
    Milanote does not sponsor video content that includes:
    - pictures or videos of animals

    Plant Guidelines
    Milanote recommends that sponsor video content includes:
    - pictures or videos of plants

    Output:
    {
    "Key points": [
        "Check that the script does not contain any pictures or videos of animals.",
        "Check that the script contains pictures or videos of plants."
        ]
    }

    """
    user_prompt_extract = f"""
    Creative Brief: {brief_guidelines}
    """

    prompt = [
            {"role": "system", "content": system_prompt_extract},
            {"role": "user", "content": user_prompt_extract}
        ]
    response_extract = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt
    )
    key_points = response_extract.choices[0].message.content.strip()
    
    return key_points

def compare_script_to_key_points(script, key_points):
    key_points_json = json.loads(key_points)
    feedbacks = []
    for key, value in key_points_json.items():
        for point in value:
            system_prompt_compare = f"""
            Guideline:{point}. 
            Keep all feedback concise and within 1 sentence. Don't hallucinate anything.
            If the script already follows the guideline, don't repeat the guideline
            """
            user_prompt_compare = f"""

            Script: {script}

            """

            prompt = [
                {"role": "system", "content": system_prompt_compare},
                {"role": "user", "content": user_prompt_compare}
            ]
            response_compare = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=prompt
            )
            feedback = response_compare.choices[0].message.content.strip()
            feedbacks.append(feedback)
    
    return feedbacks

def combine_feedback(feedbacks):
    feedback_combined_prompt = f"""
    Combine the following feedback messages into a single, cohesive feedback report. Ensure no information is repated and put it into one paragraph.

    Feedbacks:
    {json.dumps(feedbacks, indent=2)}

    Output the combined feedback.
    """
    response_combined = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": feedback_combined_prompt}]
    )

    combined_feedback = response_combined.choices[0].message.content.strip()
    
    return combined_feedback


### Method 2: extracting key points from previous feedbacks given

In [331]:
def split_feedback(feedback):
    # System message to set the behavior of the assistant
    system_prompt = (
        """
        You are a helpful assistant that separates client feedback into two sections: script feedback and video feedback.
        Your task is to process the input text and output the results in JSON format, clearly separating the two sections.

        - Script feedback: Include only feedback related to the script content (e.g., changes to lines, structure, tone, or messaging).
        - Video feedback: Include only instructions, reminders, or tips about video creation, screen recording, or other non-script-related tasks.

        If one of the sections does not have content, set it to `null`.

        Output the response as a JSON object with the following structure:

        {
            "script_feedback": "<content>",
            "video_feedback": "<content>"
        }
        """
    )
    
    user_message = (
        f"The client said: \"{feedback}\""
    )
    
    # We'll instruct the model to provide a short bullet point or sentence summarizing the key action.
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=0.0,  # zero for more deterministic output
    )
    
    json_string = response.choices[0].message.content.strip()
    splits = json.loads(json_string)
    first_word = feedback.lower().split()[0]  # Get the first word in lowercase
    decision_map = {"approved": 1, "rejected": 0}  # Define the mapping
    splits["decision"] = decision_map.get(first_word, None)

    # Extract the assistant's final answer
    return splits

In [336]:

def extract_script_key_points_2(script_feedback, decision): # feedback in structured format
    # System message to set the behavior of the assistant
    system_prompt = (
        """
        You are a helpful assistant that extracts actionable key points from the script feedback section. 
        Your task is to focus on summarizing the main instructions or requirements as a structured output, 
        organized by themes. 

        Each theme should represent the topic of the feedback, which should be a few words summarising what the feedback is about. (e.g., "Links and user engagement", "Overall value of the product", or the theme could be related to specific features of the product). 
        For each theme, list the key points as concise and actionable guidelines. Focus on the big-picture intent 
        of the feedback rather than quoting or paraphrasing specific phrases. there can be multiple themes discussed in the feedback message.

        Output the response as a structured JSON object with the following format:

        {
            "Theme 1": ["Key point 1", "Key point 2"],
            "Theme 2": ["Key point 3", "Key point 4"]
        }

        If there is only one theme, structure the output with a single key-value pair. If no feedback is provided, return an empty JSON object (`{}`).

        Example 1:

        Input:
        The script looks great and I am happy to approve it. Great work!!
        Just one small note: where you say ""But don’t take my word for it, give Milanote a go for your next project and experience it for yourself!"" can you please add a line saying something along the lines of ""I have added a link for you in the description box"". It would help to have a CTA in the middle of the video as well.

        Output:
        {
            "Links and user engagement": [
                "Mention the link in the description box.",
                "Include a CTA during the video."
            ]
        }
        
        Example 2:
        
        Input:
        Just one small note: Where you say "whether you're a crochet designer or fashion designer, Milanote makes starting a new project........", could you please change that to "whether you're a crochet designer or fashion designer or a creative in general, Milanote makes starting a new project....." because we don't want the audience to get confused and think that Milanote is just limited to fashion projects.
        
        Output:
        {
            "Milanote use cases": [
                "Don't make it seem like Milanote only caters to certain projects, as it is a tool for all creative work."
            ]
        }
        
        
        Remember:
        - Identify the theme(s) of the feedback based on the main ideas.
        - Focus on the intent, themes, or high-level suggestions behind the feedback.
        - Group actionable points under their respective themes as lists.
        - Avoid quoting specific phrases or rewording client-provided text.
        - Keep responses concise, actionable, and focused on what the reviewer needs to know.
        """
    )
    
    # The user's feedback message
    user_message = (
        f"Script feedback: \"{script_feedback}\""
    )
    
    # We'll instruct the model to provide a short bullet point or sentence summarizing the key action.
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=0.0,  # zero for more deterministic output
    )
    
    json_string = response.choices[0].message.content.strip()
    themes_and_points = json.loads(json_string)
    
    structured_output = {
        theme: {
            "key_points": points,
            "decision": decision
        }
        for theme, points in themes_and_points.items()
    }
    
    # Extract the assistant's final answer
    return structured_output

In [207]:
# Example 1:
feedback_text_full = (
"""Approved Draft Script With Feedback:
Hey Cameron, 

The script looks great and I am happy to approve it. Great work!!

Just one small note: where you say ""But don‚Äôt take my word for it, give Milanote a go for your next project and experience it for yourself!"" can you please add a line saying something along the lines of ""I have added a link for you in the description box"". It would help to have a CTA in the middle of the video as well. 

Other than that, I am happy for you to get started on your draft video. Here are some reminders for creating a great screen recording:
- Remember to show our animated logo on the screen right as you introduce Milanote.
- When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about).
- Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so that your audience can see the process of how a board is built on Milanote instead of just pointing to the board elements.
- Screen record your boards in both full-screen view and zoomed-in view. This way, we can take in your entire masterpiece while also zooming in for a closer look at specific elements.
- Show all the different features in action! For example, when you talk about the built-in templates click into the templates in your screen recording; when you talk about the web clipper tool show yourself using the tool and adding an image on your board etc.

Also, a reminder to go back to the Creative Brief for more reference. Looking forward to seeing this come to life soon and thanks for all your amazing work so far!!

Best, 
Himika

"""
)

In [208]:
# checking feedback split
splits = split_feedback(feedback_text_full)
print(splits)

{
    "script_feedback": "Just one small note: where you say \"But don‚Äôt take my word for it, give Milanote a go for your next project and experience it for yourself!\" can you please add a line saying something along the lines of \"I have added a link for you in the description box\". It would help to have a CTA in the middle of the video as well.",
    "video_feedback": "Here are some reminders for creating a great screen recording: - Remember to show our animated logo on the screen right as you introduce Milanote. - When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about). - Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so that your audience can see the process of how a board is built on Milanote instead of just pointing to the board elements. - Screen r

In [209]:

feedback = json.loads(splits)
script_feedback = feedback.get("script_feedback")
decision = feedback.get("decision")

if script_feedback:
    # Step 2: Pass script feedback to LLM for key-point extraction
    print("Script Feedback to Process: \n", script_feedback)
    print("decision:", decision)
    # Send `script_feedback` to the Step 2 prompt here.
else:
    print("No script feedback found.")

Script Feedback to Process: 
 Just one small note: where you say "But don‚Äôt take my word for it, give Milanote a go for your next project and experience it for yourself!" can you please add a line saying something along the lines of "I have added a link for you in the description box". It would help to have a CTA in the middle of the video as well.
decision: 1


the key points extracted from this feedback message:

In [210]:
key_script_point = extract_script_key_points_2(script_feedback, decision)
print(key_script_point)

{'Links and user engagement': {'key_points': ['Mention the link in the description box.', 'Include a CTA during the video.'], 'decision': 1}}


In [172]:
# example 2

feedback_text_full_2 = """
"Approved Draft Script With Feedback:
Hey Mohammed, 

Thanks for sharing your script - it looks great and I am happy to approve it :) 

Just one minor edit: instead of saying ""an interactive tool that helps you organize all the different files for your CREATIVE projects"" can you say something along the lines of ""helps you organise and visualise the planning stages of your creative projects"" 

Other than that, happy for you to put together your draft video. Here are some reminders on shooting screen-recordings for your video: 
- Remember to show our animated logo on the screen right as you introduce Milanote. You can find this in the Creative brief. 
- When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about).
- Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so that your audience can see the process of how a board is built on Milanote instead of just pointing to the board elements.
- Screen record your boards in both full-screen view and zoomed-in view. This way, we can take in your entire masterpiece while also zooming in for a closer look at specific elements.
- Show all the different features in action! For example, when you talk about the built-in templates click into the templates in your screen recording; when you talk about the web clipper tool show yourself using the tool and adding an image on your board etc.

Thanks for the great work so far and I look forward to reviewing the draft video soon!

Best, 
Himika
"
"""

In [173]:
splits = split_feedback(feedback_text_full_2)
print(splits)

{
    "script_feedback": "Just one minor edit: instead of saying \"an interactive tool that helps you organize all the different files for your CREATIVE projects\" can you say something along the lines of \"helps you organise and visualise the planning stages of your creative projects\"",
    "video_feedback": "- Remember to show our animated logo on the screen right as you introduce Milanote. You can find this in the Creative brief.\n- When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about).\n- Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so that your audience can see the process of how a board is built on Milanote instead of just pointing to the board elements.\n- Screen record your boards in both full-screen view and zoomed-in view. This way, we can take

In [174]:
feedback = json.loads(splits)
script_feedback = feedback.get("script_feedback")

if script_feedback:
    # Step 2: Pass script feedback to LLM for key-point extraction
    print("Script Feedback to Process: \n", script_feedback)
    # Send `script_feedback` to the Step 2 prompt here.
else:
    print("No script feedback found.")

Script Feedback to Process: 
 Just one minor edit: instead of saying "an interactive tool that helps you organize all the different files for your CREATIVE projects" can you say something along the lines of "helps you organise and visualise the planning stages of your creative projects"


In [175]:
key_script_point = extract_script_key_points_2(script_feedback)
print(key_script_point)

{
    "Clarifying product description": [
        "Change description to focus on planning stages of creative projects."
    ]
}


In [280]:
# test 3 for rejected script
feedback_text_full_4 = """
Rejected Draft Script With Feedback:
Hey callimara, 

Great work on the script!! I am happy to approve this as long as you make sure that you Milanote's features (e.g. boards within boards, images, notes, the ability to draw/handwrite on boards) anywhere while going through your boards. Particularly, I think the web clipper tool (https://help.milanote.com/en/articles/1015930-milanote-web-clipper-for-chrome) would be very relevant for you to talk about!

Also, an edit I want to point out in the script - where you say "so to help you up your planning game, you can also view this board and use it as a reference using the link in my description!" are you planning on adding a separate link to your board in your description box as well? If so, please make sure that you also mention your unique CTA link that will be in your description box to your audience as well. 

Again, thanks for the great work! 

Best, 
Himika
"""

In [281]:
splits = split_feedback(feedback_text_full_4)
print(splits)

{'script_feedback': "1. Consider incorporating Milanote's features like boards within boards, images, notes, and the ability to draw/handwrite on boards throughout the script. \n2. Highlight the relevance of the web clipper tool (https://help.milanote.com/en/articles/1015930-milanote-web-clipper-for-chrome) in the script. \n3. Clarify the mention of adding a separate link to the board in the description box and ensure the unique CTA link is clearly communicated to the audience.", 'video_feedback': None, 'decision': 0}


In [282]:
key_script_point = extract_script_key_points_2(splits["script_feedback"], splits["decision"])
print(key_script_point)

{'Milanote features': {'key_points': ['Incorporate features like boards within boards, images, notes, and drawing/handwriting on boards throughout the script.'], 'decision': 0}, 'Web clipper tool': {'key_points': ['Highlight the relevance of the Milanote web clipper tool in the script.'], 'decision': 0}, 'Links and user engagement': {'key_points': ['Clarify the mention of adding a separate link to the board in the description box.', 'Ensure the unique CTA link is clearly communicated to the audience.'], 'decision': 0}}


In [306]:
# test 5: no script feedback
feedback_text_full_5 = "Approved Draft Script"

In [308]:
splits = split_feedback(feedback_text_full_5)
print(splits)

{'script_feedback': None, 'video_feedback': None, 'decision': 1}


consolidating feedback key points

In [338]:
def consolidate_feedbacks(feedback_list):
    consolidated_feedback = {}
    
    for i, feedback in enumerate(feedback_list):
        splits = split_feedback(feedback)
        
        
        if script_feedback:
            key_points = extract_script_key_points_2(splits["script_feedback"], splits["decision"])
            # Consolidate results by adding themes and key points to the dictionary
            for theme, data in key_points.items():
                if theme not in consolidated_feedback:
                    consolidated_feedback[theme] = {
                        "key_points": [],
                        "decision": data["decision"] 
                    }
                # Merge key points, avoiding duplicates
                consolidated_feedback[theme]["key_points"].extend(data["key_points"])
                consolidated_feedback[theme]["key_points"] = list(set(consolidated_feedback[theme]["key_points"]))
  
    return consolidated_feedback



summarising the consolidated feedback key points

In [312]:
import json

def summarize_feedback(consolidated_feedback):
    # Convert consolidated feedback into JSON-like string for the LLM prompt
    feedback_input = json.dumps(consolidated_feedback, indent=4)
    
    # LLM prompt
    system_prompt = """
    You are a helpful assistant that summarizes feedback by identifying and merging similar points into a concise summary. 
    Your task is to analyze a list of key feedback points across multiple themes, remove repetitions, and combine similar ideas.
    Maintain the same JSON structure as the original input, which is specified below:
    
    {
        "Theme 1": {
            "key_points": ["Key point 1", "Key point 2"],
            "decision": 0
        },
        "Theme2": {
            "key_points": ["Key point 3", "Key point 4"],
            "decision": 1
        }
    }
    
    Pay attention to the decision value in each theme. If there are similar themes with different decisions, set the decision to 0.
    """
    user_message = feedback_input
    
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=0.0,  # zero for more deterministic output
    )
    
    # Parse the response into a dictionary
    output = response.choices[0].message.content.strip()
    # print(output)   
    # print()
    summarized_feedback = json.loads(output)
    
    return summarized_feedback



choosing a mix of accepted and rejected scripts for feedback consolidation and summarization

In [301]:
rejected_scripts = data_scripts[data_scripts["decision"] == 0]
accepted_scripts = data_scripts[data_scripts["decision"] == 1]


In [332]:
# Randomly sample 10 feedback messages from accepted_scripts
sampled_accepted_feedback = accepted_scripts["message"].sample(n=5, random_state=42).tolist()

# Randomly sample 5 feedback messages from rejected_scripts
sampled_rejected_feedback = rejected_scripts["message"].sample(n=2, random_state=42).tolist()

# Combine them into a single list
feedback_list = sampled_accepted_feedback + sampled_rejected_feedback

print("Feedback List:")
feedback_list

Feedback List:


["Approved Draft Script With Feedback:\nHey Elizabeth, \n\nYour direction with the script is absolutely perfect and the example you have given for your screen recording voiceover sounds great! \n\nJust a reminder to make sure that you explain Milanote's features by interacting with and explaining your room makeover project because keeping in mind that your integration is 5 minutes long, sometimes it can sound like a Milanote tutorial :)\n\nPlease feel free to get started on your draft video. Here are our top tips for creating a great screen recording: \n- Remember to show our animated logo on the screen right as you introduce Milanote. \n- When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about). \n- Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so that your 

In [ ]:
# consolidate feedback
consolidated_feedback = consolidate_feedbacks(feedback_list)


In [340]:
# summarize feedback
summarized_feedback = summarize_feedback(consolidated_feedback)

In [341]:
print(json.dumps(summarized_feedback, indent=4))

{
    "Milanote features demonstration": {
        "key_points": [
            "Avoid making the integration sound like a Milanote tutorial.",
            "Interact with Milanote features during the room makeover project."
        ],
        "decision": 1
    },
    "Audience engagement and value proposition": {
        "key_points": [
            "Explain how the audience can use Milanote for planning art projects, gathering inspiration, collecting references, and creating mood boards."
        ],
        "decision": 1
    },
    "Overall script feedback": {
        "key_points": [
            "Client approves the script for filming.",
            "Emphasize showcasing Milanote's value effectively."
        ],
        "decision": 1
    },
    "Content detail and engagement": {
        "key_points": [
            "Add more detail on specific features or benefits of Milanote to engage the audience further."
        ],
        "decision": 1
    },
    "Technical issues": {
        "key_p

### last stage of pipeline: generating script feedback based on the script and key points

In [377]:
def generate_feedback_2(script, summarized_feedback):
    """
    Generate feedback for a video script using an LLM based on consolidated feedback key points.
    
    Args:
        script (str): The video script to evaluate.
        summarised_feedback (dict): Consolidated and summarised feedback with key points and decision values.

    Returns:
        str: Feedback report for the script.
    """
    # LLM system prompt
    system_prompt = """
    You are a helpful assistant that reviews video scripts and provides feedback based on a set of feedback key points. 
    Your task is to:
    1. Analyze the video script and identify any relevant topics that match the provided feedback key points.
    2. If a key point does not match any part of the script, do not talk about that point. 
    3. Add actionable feedback only for areas where the script does not align with the feedback key points.
    4. Do not talk about more than 2 themes. The feedback needs to be concise

    Output the feedback in the following format:
    "Draft Script with Further Feedback"
    <insert feedback about the script here>

    """

    # Prepare user message
    user_message = f"""
    Script:
    {script}

    Feedback Key Points:
    {json.dumps(summarized_feedback, indent=4)}
    """

    # Prepare the LLM prompt
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]

    # Call the LLM API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=prompt,
        temperature=0.0,  # Zero for deterministic output
    )

    # Extract the LLM's response
    return response.choices[0].message.content.strip()

generated feedback:

In [378]:
test_script = accepted_scripts.iloc[15]["input"]
output = generate_feedback_2(test_script, summarized_feedback)
print(output)

Draft Script with Further Feedback
The script does a good job of mentioning Milanote and its features, but it could benefit from more specific details on how Milanote can be used for planning art projects, gathering inspiration, collecting references, and creating mood boards. Adding more depth on these aspects would engage the audience further and showcase Milanote's value more effectively. Consider incorporating more specific examples or scenarios where Milanote can enhance the creative process for artists.


actual feedback and script

In [343]:
print(accepted_scripts.iloc[15]["input"])


Every now and then I like to compare my old art and the art I make currently to see if I’m improving, and also critic myself and find areas in my art I can improve on and get better. But it’s usually difficult organizing every drawing together and putting side by side on the same page so i reached out to the heavens for help and the art gods blessed me with milanote, the sponsor of today’s video. Milanote is an interactive tool that helps you organize all the different files for your CREATIVE projects on one platform so accessing them can be easier. Milanote allows you to collect and organize pictures, videos, notes and text, and arrange them into different boards using any of their built in templates that make it easy to start off your creative projects. You can also create several tasks for different things you want to study, and tick them when you’re done. For this instance, I’ll start by creating a general board where I’ll layout a broad overview that covers everything concerning t

In [323]:
print(accepted_scripts.iloc[15]["message"])

Approved Draft Script With Feedback:
Hey Mohammed, 

Thanks for sharing your script - it looks great and I am happy to approve it :) 

Just one minor edit: instead of saying "an interactive tool that helps you organize all the different files for your CREATIVE projects" can you say something along the lines of "helps you organise and visualise the planning stages of your creative projects" 

Other than that, happy for you to put together your draft video. Here are some reminders on shooting screen-recordings for your video: 
- Remember to show our animated logo on the screen right as you introduce Milanote. You can find this in the Creative brief. 
- When introducing Milanote at the beginning of the segment, jump right into displaying Milanote boards on screen (We recommend showing your project board since that's what folks are most excited about).
- Get interactive with the great boards you have built. Delete some content from your boards and add it back in while screen recording so t